Libraries installation + imports

In [ ]:
!pip install h5py
!pip install pillow

In [ ]:
import h5py
import matplotlib.pyplot as plt
import os
import random
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
import pandas as pd
import kagglehub
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm
from torch.utils.data import DataLoader
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import time

Download Kaggle dataset(uplod kaggle.json) too

In [ ]:
!mkdir -p ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Donwload to the Google drive directory

In [ ]:
!kaggle competitions download -c isic-2024-challenge -p /content/drive/MyDrive/ML_project/kaggle

In [ ]:
project_dir = '/content/drive/MyDrive/ML_project/kaggle'
!unzip  "{project_dir}/isic-2024-challenge.zip" -d /content/isic-2024-challenge;

In [ ]:
root_dir = '/content/isic-2024-challenge'

Explore test dataset

In [ ]:
with h5py.File(f'{root_dir}/test-image.hdf5', 'r') as f:
    print("Keys in the file:", list(f.keys()))

    keys = list(f.keys())
    for key in keys:
        dataset = f[key]
        print(f"\nInspecting '{key}':")

        if dataset.shape == ():
            print(f"'{key}' is a scalar with value: {dataset[()]}")
        else:
            print(f"'{key}' is an array with shape: {dataset.shape}")
            plt.show()

Visualize the train images

In [ ]:
image_dir = f'{root_dir}/train-image/image'

all_images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

random_images = random.sample(all_images, 10)

plt.figure(figsize=(15, 10))
for i, img_name in enumerate(random_images):
    img_path = os.path.join(image_dir, img_name)
    img = Image.open(img_path)

    plt.subplot(2, 5, i+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(img_name)

plt.tight_layout()
plt.show()


Hair removal algorithm

In [ ]:
def remove_hair(image):
    """
    Remove hair artifacts from an image using the DullRazor approach.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, thresh = cv2.threshold(blackhat, 12, 255, cv2.THRESH_BINARY)
    inpainted = cv2.inpaint(image, thresh, inpaintRadius=1, flags=cv2.INPAINT_TELEA)

    return inpainted

To check resutls of hair removal alforithm

In [ ]:
def preprocess_image(image_path, output_size=(224, 224)):
    """
    Preprocess an image by removing hair, resizing, and normalizing.
    Also visualizes the image before and after hair removal.
    """
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image)
    hair_removed_image = remove_hair(image_np)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image_np)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(hair_removed_image)
    plt.title("After Hair Removal")
    plt.axis('off')

    plt.show()

    hair_removed_image = Image.fromarray(hair_removed_image)

    preprocess_transform = transforms.Compose([
        transforms.Resize(output_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image_tensor = preprocess_transform(hair_removed_image)

    return image_tensor

image_path = f'{root_dir}/train-image/image/ISIC_2314445.jpg'
preprocessed_image = preprocess_image(image_path)

Test dataset

In [ ]:
data = pd.read_csv(f'{root_dir}/train-metadata.csv')
selected_data = data[['isic_id', 'target']]

Split the data that we take all present malignant data and only 5K of benign data

In [ ]:
data_target_1 = selected_data[selected_data['target'] == 1]

num_samples_target_0 = 5000
data_target_0 = selected_data[selected_data['target'] == 0].sample(n=num_samples_target_0, random_state=42)
balanced_data = pd.concat([data_target_1, data_target_0], axis=0).reset_index(drop=True)

print(balanced_data['target'].value_counts())

Add [ISIC-2020 dataset:](https://www.kaggle.com/datasets/nischaydnk/isic-2020-jpg-256x256-resized)

In [ ]:
path = kagglehub.dataset_download("fanconic/skin-cancer-malignant-vs-benign")
print("Path to dataset files:", path)

Move downloaded dataset to desired location

In [ ]:
!mv /root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4 /content/isic-2024-challenge/isic-2020-dataset

Add new data to our dataset(only malignant images)

In [ ]:
file_list = os.listdir('/content/isic-2024-challenge/isic-2020-dataset/4/train/malignant/')

isic_2020_dataset = pd.DataFrame({
    'isic_id': ['ISIC2020_' + file for file in file_list],
    'target': 1
})

In [ ]:
merged_dataset = pd.concat([balanced_data, isic_2020_dataset], ignore_index=True)

Depicts data distribution

In [ ]:
value_counts = merged_dataset.target.value_counts()
plt.figure(figsize=(10, 6))
bars = plt.bar(value_counts.index, value_counts.values, color=['#1f77b4', '#ff7f0e'], edgecolor='black')
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 5, int(yval), ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.title("Distribution of Target Values", fontsize=16, fontweight='bold')
plt.xlabel("Target", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.xticks([0, 1], labels=['Class 0', 'Class 1'], fontsize=12)
plt.show()


Split the data in the equal proportions

In [ ]:
train_data, val_data = train_test_split(
    merged_dataset, test_size=0.2, stratify=merged_dataset['target'], random_state=42
)

Data Loader for our dataset

In [ ]:
class CassavaDataset(torch.utils.data.Dataset):
    """
    Helper Class to create the PyTorch dataset
    """

    def __init__(self, df, data_path='isic-2024-challenge/train-image/image/', secondary_data_path="/content/isic-2024-challenge/isic-2020-dataset/4/train/malignant", mode="train", transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.secondary_data_path = secondary_data_path
        self.transforms = transforms
        self.mode = mode

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name, label = self.df_data[index]
        if img_name.startswith("ISIC2020_"):
            stripped_name = img_name.replace("ISIC2020_", "")
            img_path = os.path.join(self.secondary_data_path, f"{stripped_name}")
        else:
            img_path = os.path.join(self.data_path, f"{img_name}.jpg")
        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            img = self.transforms(img)

        return img, label


Dataset Preprocessing

In [ ]:
class HairRemovalTransform:
    def __call__(self, img):
        img_np = np.array(img)
        img_np = remove_hair(img_np)
        img = Image.fromarray(img_np)
        return img

IMG_SIZE = 224

transforms_train = transforms.Compose(
    [
        HairRemovalTransform(),
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomResizedCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

transforms_valid = transforms.Compose(
    [
        HairRemovalTransform(),
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)


ViT model training

In [ ]:
class ViTBase16(nn.Module):
    def __init__(self, n_classes, pretrained=False, model_path=None):
        super(ViTBase16, self).__init__()
        self.model = timm.create_model("vit_base_patch16_224", pretrained=False)

        if pretrained and model_path:
            self.model.load_state_dict(torch.load(model_path))
        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return torch.sigmoid(x)

    def train_one_epoch(self, train_loader, criterion, optimizer, device):
        self.model.train()
        epoch_loss = 0.0
        epoch_accuracy = 0.0

        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = self(data)
            loss = criterion(output, target)
            loss.backward()
            accuracy = (output.argmax(dim=1) == target).float().mean()

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()

            optimizer.step()
            if device.type == "xla" and i % 20 == 0:
                xm.master_print(f"\tBATCH {i+1}/{len(train_loader)} - LOSS: {loss.item():.4f}")

        avg_loss = epoch_loss / len(train_loader)
        avg_accuracy = epoch_accuracy / len(train_loader)
        return avg_loss, avg_accuracy

    def validate_one_epoch(self, valid_loader, criterion, device):
        self.model.eval()
        valid_loss = 0.0
        valid_accuracy = 0.0

        with torch.no_grad():
            for data, target in valid_loader:
                data, target = data.to(device), target.to(device)

                output = self(data)
                loss = criterion(output, target)

                accuracy = (output.argmax(dim=1) == target).float().mean()

                valid_loss += loss.item()
                valid_accuracy += accuracy.item()

        avg_loss = valid_loss / len(valid_loader)
        avg_accuracy = valid_accuracy / len(valid_loader)
        return avg_loss, avg_accuracy


In [ ]:
train_dataset = CassavaDataset(train_data, data_path='isic-2024-challenge/train-image/image/', transforms=transforms_train)
valid_dataset = CassavaDataset(val_data, data_path='isic-2024-challenge/train-image/image/', transforms=transforms_valid)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

Model training

In [ ]:
model = ViTBase16(n_classes=2, pretrained=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies  = []

num_epochs = 10
best_valid_acc = 0.0
total_time = 0

for epoch in range(num_epochs):
    start_time = time.time()

    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 20)
    train_loss, train_acc = model.train_one_epoch(train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    print(f"Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

    valid_loss, valid_acc = model.validate_one_epoch(valid_loader, criterion, device)
    valid_losses.append(valid_loss)
    valid_accuracies.append(valid_acc)
    print(f"Validation: Loss = {valid_loss:.4f}, Accuracy = {valid_acc:.4f}")

    if valid_acc > best_valid_acc:
        print(f"Validation accuracy improved from {best_valid_acc:.4f} to {valid_acc:.4f}. Saving model...")
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), "best_model.pth")

    epoch_time = time.time() - start_time
    total_time += epoch_time
    avg_epoch_time = total_time / (epoch + 1)
    time_left = avg_epoch_time * (num_epochs - epoch - 1)

    print(f"Time for epoch {epoch+1}: {epoch_time:.2f} seconds")
    print(f"Estimated time remaining: {time_left:.2f} seconds")

    print(f"\nEpoch Summary: {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}")
    print(f"Valid Loss: {valid_loss:.4f} | Valid Accuracy: {valid_acc:.4f}")
    print(f"Best Validation Accuracy so far: {best_valid_acc:.4f}\n")
